In [160]:
from pysolar.solar import *
import datetime
import pytz
import pyvista as pv
import numpy as np
#load geometry
plane = pv.read('geometry/plane.stl')
box = pv.read('geometry/box.stl')
central_points = plane.cell_centers()

lat,long = [43.48574132496213, -80.48384643009003]
#2023 07 01, 9:00am timezone = est
est = pytz.timezone('US/Eastern')
hours = np.arange(0,23,1)
minutes = np.array([0,30])
total_rec = 0
bright_field = np.zeros(central_points.points.shape[0])
for i_h, hr_ in enumerate(hours):
    for i_m, min_ in enumerate(minutes):
        est_time = est.localize(datetime.datetime(2023, 7, 1, hr_, min_, 0))  # Replace with your desired EST time
        # Convert EST time to UTC
        utc_time = est_time.astimezone(pytz.utc)
        #get sun location
        d_sun = 1.5e11
        alt_deg = get_altitude(lat, long, utc_time)
        az_deg = get_azimuth(lat, long, utc_time)

        sun_coord = np.array([np.cos(np.deg2rad(az_deg)) * d_sun, 
                             np.sin(np.deg2rad(az_deg)) * d_sun, 
                             np.sin(np.deg2rad(alt_deg)) * d_sun])
        #print('Sun coordinate: {}'.format(sun_coord))
        cell_ind = []
        #print('Radiation: {}'.format(radiation.get_radiation_direct(utc_time, alt_deg)))
        if radiation.get_radiation_direct(utc_time, alt_deg) > 200:
            total_rec += 1
            for i,point in enumerate(central_points.points):
                points, ind = box.ray_trace(point, sun_coord)
                if len(points) == 0:
                    bright_field[i] += 1
        
central_points['sunlite'] = bright_field

In [161]:
p = pv.Plotter()
p.add_mesh(box)
p.add_mesh(central_points,scalars='sunlite',clim=[0,20])
p.view_xy()
p.show()

Widget(value="<iframe src='http://localhost:58104/index.html?ui=P_0x29e0eac74d0_19&reconnect=auto' style='widt…